In [ ]:
%%sh
pip install -q sagemaker awscli --upgrade

In [ ]:
import sagemaker

sess = sagemaker.Session()
role = sagemaker.get_execution_role()

## Download the Fashion-MNIST dataset

In [ ]:
import os
import keras
import numpy as np
from keras.datasets import fashion_mnist
(x_train, y_train), (x_val, y_val) = fashion_mnist.load_data()

os.makedirs("./data", exist_ok = True)

np.savez('./data/training', image=x_train, label=y_train)
np.savez('./data/validation', image=x_val, label=y_val)

## Upload Fashion-MNIST data to S3

In [ ]:
prefix = 'keras-fashion-mnist'

training_input_path   = sess.upload_data('data/training.npz', key_prefix=prefix+'/training')
validation_input_path = sess.upload_data('data/validation.npz', key_prefix=prefix+'/validation')

print(training_input_path)
print(validation_input_path)

## Configure the training job on a GPU instance

In [ ]:
!pygmentize mnist_keras_tf.py

In [ ]:
from sagemaker.tensorflow import TensorFlow

tf_estimator = TensorFlow(entry_point='mnist_keras_tf.py', 
                          role=role,
                          train_instance_count=1, 
                          train_instance_type='ml.p3.2xlarge',
                          framework_version='1.15', 
                          py_version='py3',
                          script_mode=True
                         )

In [ ]:
tf_estimator.fit({'training': training_input_path, 'validation': validation_input_path})

In [ ]:
print(tf_estimator.model_data)

In [ ]:
%env model_data {tf_estimator.model_data}

# Now... how about deploying that model to AWS Fargate?

## Get model artefact and push it to Git repository

In [ ]:
%%sh
aws s3 cp ${model_data} .
mkdir test-models
tar xvfz model.tar.gz -C test-models 

In [ ]:
# cd test-models
# git add model
# git commit -m 'New model'
# git push

## Install ecs-cli

In [ ]:
%%sh
sudo curl -o /usr/local/bin/ecs-cli https://amazon-ecs-cli.s3.amazonaws.com/ecs-cli-linux-amd64-latest
sudo chmod 755 /usr/local/bin/ecs-cli

## Create cluster

In [ ]:
%%sh 

# Make sure your IAM role includes ecs:CreateCluster
aws ecs create-cluster --cluster-name fargate-demo
# ecs-cli : https://github.com/aws/amazon-ecs-cli
ecs-cli configure --cluster fargate-demo --region eu-west-1

In [ ]:
%%sh

ecs-cli ps --desired-status RUNNING

## Create log group

In [ ]:
%%sh
# You only need to do this once
aws logs create-log-group --log-group-name awslogs-tf-ecs

## Run inference task

In [ ]:
%%sh

# You only need to do this when you create and update the task definition
aws ecs register-task-definition --cli-input-json file://inference-fargate-tf115-sagemaker.json

In [ ]:
%%sh

# Update these with your own settings
export SECURITY_GROUP_ID=sg-0504d9aef33f34caf # SSH + Tensorflow Serving ports
export SUBNET_ID=subnet-cbf5bdbc              # Any subnet in your default VPC will do

aws ecs run-task --cluster fargate-demo --task-definition inference-fargate-tf115:6 --count 1 \
    --launch-type FARGATE \
    --network-configuration "awsvpcConfiguration={subnets=[$SUBNET_ID], \
                            securityGroups=[$SECURITY_GROUP_ID], \
                            assignPublicIp=ENABLED}"

In [ ]:
%%sh

ecs-cli ps --desired-status RUNNING

## Predict

In [ ]:
inference_task_ip = '52.49.238.243'
inference_url = 'http://'+inference_task_ip+':8501/v1/models/1:predict'

In [ ]:
import random

num_samples = 10
indices = random.sample(range(x_val.shape[0] - 1), num_samples)
images = x_val[indices]/255
labels = y_val[indices]

data = images.reshape(num_samples, 28, 28, 1)

In [ ]:
import json, requests

headers = {"content-type": "application/json"}
data    = json.dumps({"signature_name": "serving_default", "instances": data.tolist()})

json_response = requests.post(inference_url, data=data, headers=headers)

predictions = json.loads(json_response.text)['predictions']
print(predictions)
predictions = np.array(predictions).argmax(axis=1)

print("Labels     : ", labels)
print("Predictions: ", predictions)

In [ ]:
%%sh
aws ecs stop-task --cluster fargate-demo --task arn:aws:ecs:eu-west-1:613904931467:task/a9c9a3a8-8b7c-4dbb-9ec4-d20686ba5aec

In [ ]:
%%sh
ecs-cli down -f --cluster fargate-demo 